In [25]:
from fyers_apiv3 import fyersModel
from fyers_apiv3.FyersWebsocket import data_ws

import pandas as pd
import numpy as np
import math

import datetime as dt
from datetime import date, timedelta,datetime
import time 

from fyers_apiv3.FyersWebsocket import data_ws
import csv

import matplotlib.pyplot as plt
import mplfinance as mpf

import scipy

from  time import sleep
import threading
import os
import pyotp
import requests
import json
import pytz
from urllib.parse import parse_qs,urlparse
import base64

import warnings
pd.set_option('display.max_columns', None)
warnings.filterwarnings('ignore')

{'ltp': 5962.0, 'symbol': 'MCX:CRUDEOILM24OCTFUT', 'type': 'sf'} 	time:  23 6 29
Cause Candle: 23:04:00
Affect Candle: 23:06:00
Inside Candle (IC) activated. Mother candle high: 5962 low: 5960
IC Flag: 1, Active Trade: 0, Trade Count: 0



In [2]:
redirect_uri = "https://trade.fyers.in/api-login/redirect-uri/index.html"
client_id='HTHFWXFS49-100'
secret_key = 'JOQVVTY9UJ'
FY_ID = "YB00632"  
TOTP_KEY = "3Y7PAJSPNHXEADGRDD726MLHJW43AJGH"  
PIN = "1964" 

grant_type = "authorization_code"                  
response_type = "code"                         
state = "sample"                                  

appSession = fyersModel.SessionModel(client_id = client_id, redirect_uri = redirect_uri,response_type=response_type,state=state,secret_key=secret_key,grant_type=grant_type)


generateTokenUrl = appSession.generate_authcode()
generateTokenUrl


def getEncodedString(string):
    string = str(string)
    base64_bytes = base64.b64encode(string.encode("ascii"))
    return base64_bytes.decode("ascii")
  



URL_SEND_LOGIN_OTP="https://api-t2.fyers.in/vagator/v2/send_login_otp_v2"
res = requests.post(url=URL_SEND_LOGIN_OTP, json={"fy_id":getEncodedString(FY_ID),"app_id":"2"}).json()   

if datetime.now().second % 30 > 27 : sleep(5)
URL_VERIFY_OTP="https://api-t2.fyers.in/vagator/v2/verify_otp"
res2 = requests.post(url=URL_VERIFY_OTP, json= {"request_key":res["request_key"],"otp":pyotp.TOTP(TOTP_KEY).now()}).json()  


ses = requests.Session()
URL_VERIFY_OTP2="https://api-t2.fyers.in/vagator/v2/verify_pin_v2"
payload2 = {"request_key": res2["request_key"],"identity_type":"pin","identifier":getEncodedString(PIN)}
res3 = ses.post(url=URL_VERIFY_OTP2, json= payload2).json()  


ses.headers.update({
    'authorization': f"Bearer {res3['data']['access_token']}"
})


TOKENURL="https://api-t1.fyers.in/api/v3/token"
payload3 = {"fyers_id":FY_ID,
           "app_id":client_id[:-4],
           "redirect_uri":redirect_uri,
           "appType":"100","code_challenge":"",
           "state":"None","scope":"","nonce":"","response_type":"code","create_cookie":True}

res3 = ses.post(url=TOKENURL, json= payload3).json()  


url = res3['Url']
parsed = urlparse(url)
auth_code = parse_qs(parsed.query)['auth_code'][0]


grant_type = "authorization_code" 

response_type = "code"  

session = fyersModel.SessionModel(
    client_id=client_id,
    secret_key=secret_key, 
    redirect_uri=redirect_uri, 
    response_type=response_type, 
    grant_type=grant_type
)

session.set_token(auth_code)

response = session.generate_token()

access_token = response['access_token']

fyers = fyersModel.FyersModel(client_id=client_id, is_async=False, token=access_token, log_path=os.getcwd())
fyers.get_profile()

{'s': 'ok',
 'code': 200,
 'message': '',
 'data': {'fy_id': 'YB00632',
  'name': 'BABAN MARUTI PAWAR',
  'image': None,
  'display_name': None,
  'pin_change_date': '21-12-2023 21:01:26',
  'email_id': 'babanpawar8975@gmail.com',
  'pwd_change_date': None,
  'PAN': '---------',
  'mobile_number': '8975963982',
  'totp': True,
  'pwd_to_expire': 90}}

In [3]:
start_date = date.today()
end_date = start_date
print("Start Date:", start_date)
print("End Date:", end_date)

Start Date: 2024-10-21
End Date: 2024-10-21


In [4]:
sym = 'MCX:CRUDEOILM24OCTFUT' #'NSE:NIFTY50-INDEX'

In [5]:
historydata = pd.DataFrame()

In [33]:
def gethistorydata(st,ed,res):
    data = {
            "symbol":str(sym),
            "resolution":str(res),
            "date_format":"1",
            "range_from":st,
            "range_to":ed,
            "cont_flag":"0"
            }
    repo = fyers.history(data=data)
#     print(repo)
    data = pd.DataFrame.from_dict(repo['candles'])
    cols = ['datetime', 'open', 'high','low','close', 'volume']
    data.columns = cols
    
    data['datetime'] = pd.to_datetime(data['datetime'],unit="s")
    data['datetime'] = data['datetime'].dt.tz_localize('utc').dt.tz_convert('Asia/Kolkata')
    global historydata
#     data = data.set_index('datetime')
    historydata = data
    historydata['datetime'] = pd.to_datetime(historydata['datetime'])

    historydata['date'] = historydata['datetime'].dt.date
    historydata['time'] = historydata['datetime'].dt.time

    #historydata.drop(columns=['datetime'], inplace=True)
    historydata.drop(columns=['volume'], inplace=True)
    
    #append(historydata,data)],axis=0)
    #historydata.to_csv("75op.csv")
    #print(historydata)

In [34]:
#gethistorydata(start_date,end_date,1)

In [35]:
def plot_candles():
    mpf.plot(historydata, type='candle', style='yahoo', title='Candlestick Chart', ylabel='Price')

In [42]:
ic_flag= 0
active_trade= 0
trade_count=0
ce = 0
strike_price=0
cause_candle = None
affect_candle = None
mother = None
tg=0
sl=0
rrr=0
entrypoint=0

In [43]:
def onmessage(message):
    print(message,"\ttime: ",time.localtime().tm_hour,time.localtime().tm_min,time.localtime().tm_sec)
    # time.sleep(1)
    global ic_flag, active_trade, trade_count, ce, strike_price,cause_candle,affect_candle,mother,tg,rrr,sl,entrypoint
    
    gethistorydata(start_date, end_date, 1)
    # print(historydata)

    if(ic_flag != 1):
        cause_candle = historydata.iloc[-2]  
        affect_candle = historydata.iloc[-1] 
    
        # Logging to check the last two candles for debugging purposes
        print("Cause Candle:", cause_candle['time'])
        print("Affect Candle:", affect_candle['time'])
    
    # Step 1: Identify if the affect candle is a baby candle (inside candle)
    if (cause_candle['high'] > affect_candle['high'] and cause_candle['low'] < affect_candle['low'] and ic_flag == 0):
        # The affect candle is within the high/low of the cause candle
        ic_flag = 1  # Mark that the inside candle condition is met
        mother = cause_candle  # Store the mother candle for later comparisons
        print("Inside Candle (IC) activated. Mother candle high:", mother['high'], "low:", mother['low'])
        plot_candles()
        
    # Step 2: Check for trade entry when a candle breaks out of the mother candle's high or low
    if (ic_flag == 1 and active_trade == 0 and trade_count < 4):
        # If no active trade and an inside candle pattern exists, check for breakout conditions
    
        # Buy condition (Call side): if the current price crosses the mother's high
        if message['ltp'] > mother['high'] :
            entrypoint = message['ltp']
            sl = historydata['close'].iloc[-1]  # SL is the close of the breakout candle
            tg = (entrypoint - sl) + entrypoint  # Target 1:1 RR
            print(f"Call trade activated at {message['ltp']}. SL: {sl}, Target: {tg}")
            active_trade = 1
            trade_count += 1
            ce=1
    
        # Sell condition (Put side): if the current price crosses the mother's low
        elif message['ltp'] < mother['low']:
            entrypoint = message['ltp']
            sl = historydata['close'].iloc[-1]  # SL is the close of the breakout candle
            tg = entrypoint - (sl - entrypoint)  # Target 1:1 RR
            print(f"Put trade activated at {message['ltp']}. SL: {sl}, Target: {tg}")
            active_trade = 1
            trade_count += 1
            ce=0

        else:
            ic_flag = 0
    
    # Step 3: Exit conditions for an active trade
    if ic_flag == 1 and active_trade == 1:
        # Check if the target or stop-loss is hit
        if ce == 1:  # Call side (price going up)
            if message['ltp'] >= tg:
                print(f"Target hit! Exiting trade at {message['ltp']}")
                active_trade = 0
            elif message['ltp'] <= sl:
                print(f"Stop-loss hit! Exiting trade at {message['ltp']}")
                active_trade = 0
    
        elif ce == 0:  # Put side (price going down)
            if message['ltp'] <= tg:
                print(f"Target hit! Exiting trade at {message['ltp']}")
                active_trade = 0
            elif message['ltp'] >= sl:
                print(f"Stop-loss hit! Exiting trade at {message['ltp']}")
                active_trade = 0

    # Additional logging to track variables during execution
    print(f"IC Flag: {ic_flag}, Active Trade: {active_trade}, Trade Count: {trade_count}\n")


def onerror(message):
    print("Error:", message)


def onclose(message):
    print("Connection closed:", message)


def onopen():
    data_type = "SymbolUpdate"

    symbols = [sym]
    wfyers.subscribe(symbols=symbols, data_type=data_type)
    
    wfyers.keep_running()


waccess_token = client_id+":"+access_token 

wfyers = data_ws.FyersDataSocket(
    access_token=waccess_token,       # Access token in the format "appid:accesstoken"
    log_path="",                     # Path to save logs. Leave empty to auto-create logs in the current directory.
    litemode=True,                  # Lite mode disabled. Set to True if you want a lite response.
    write_to_file=False,              # Save response in a log file instead of printing it.
    reconnect=True,                  # Enable auto-reconnection to WebSocket on disconnection.
    on_connect=onopen,               # Callback function to subscribe to data upon connection.
    on_close=onclose,                # Callback function to handle WebSocket connection close events.
    on_error=onerror,                # Callback function to handle WebSocket errors.
    on_message=onmessage            # Callback function to handle incoming messages from the WebSocket.
)

wfyers.connect()

{'type': 'cn', 'code': 200, 'message': 'Authentication done', 's': 'ok'} 	time:  23 34 43
{'type': 'cn', 'code': 200, 'message': 'Authentication done', 's': 'ok'} 	time:  23 34 43
Cause Candle: 23:28:00
Affect Candle: 23:29:00
IC Flag: 0, Active Trade: 0, Trade Count: 0

Cause Candle: 23:28:00
Affect Candle: 23:29:00
IC Flag: 0, Active Trade: 0, Trade Count: 0

{'type': 'lit', 'code': 200, 'message': 'Lite Mode On', 's': 'ok'} 	time:  23 34 44
{'type': 'lit', 'code': 200, 'message': 'Lite Mode On', 's': 'ok'} 	time:  23 34 44
Cause Candle: 23:28:00
Affect Candle: 23:29:00
IC Flag: 0, Active Trade: 0, Trade Count: 0

Cause Candle: 23:28:00
Affect Candle: 23:29:00
IC Flag: 0, Active Trade: 0, Trade Count: 0

{'type': 'sub', 'code': 200, 'message': 'Subscribed', 's': 'ok'} 	time:  23 34 47
Cause Candle: 23:28:00
Affect Candle: 23:29:00
IC Flag: 0, Active Trade: 0, Trade Count: 0

{'ltp': 5940.0, 'symbol': 'MCX:CRUDEOILM24OCTFUT', 'type': 'sf'} 	time:  23 34 47
Cause Candle: 23:28:00
Affec